In [1]:
import numpy as np 
import torch 
import os 

os.environ['CUDA_VISIBLE_DEVICES']='0'
device = torch.device('cpu')

In [2]:
import pickle
import random 
import time 

In [5]:
from utils_dataset import sample_nan, fill_panel, sample_nan_pos

In [4]:
data_dir = 'data/20240308_data/'

r_all_M7 = np.load(data_dir+'r_list_M7.npy', allow_pickle=True)
with open(data_dir+'r_dict_M7.pkl', 'rb') as file:
    r_dict_M7 = pickle.load(file)
with open(data_dir+'r_last_M7.pkl', 'rb') as file:
    r_last_M7 = pickle.load(file)
    
r_all_M10 = np.load(data_dir+'r_list_M10.npy', allow_pickle=True)
with open(data_dir+'r_dict_M10.pkl', 'rb') as file:
    r_dict_M10 = pickle.load(file)
with open(data_dir+'r_last_M10.pkl', 'rb') as file:
    r_last_M10 = pickle.load(file)
    
with open(data_dir+'r_dict_pos.pkl', 'rb') as file:
    r_dict_pos = pickle.load(file)
    
with open(data_dir+'r_dict_num.pkl', 'rb') as file:
    r_dict_num = pickle.load(file)
    
with open(data_dir+'r_dict_pos_withR.pkl', 'rb') as file:
    r_dict_pos_withR = pickle.load(file)

r_dict_num_keylist = list(r_dict_num.keys())
r_list_num = np.load(data_dir+'r_list_num.npy', allow_pickle=True)
num_samples_total = 12100

In [8]:
i_a = 3
for i_R in range(7): 
    
    r_list = r_list_num[i_R]
    num_samples_per_r = int(np.ceil(num_samples_total/len(r_list)))

    attr_all = []

    for i_r in range(len(r_list)):

        start_time = time.time()

        for i_sample in range(num_samples_per_r):

            x_num = r_list[i_r] # e.g., [[2], [2], [2]]

            x_shape = sample_x_with_num(x_num, r_all_M7, r_dict_M7, r_last_M7)
            x_size = sample_x_with_num(x_num, r_all_M10, r_dict_M10, r_last_M10)
            x_color = sample_x_with_num(x_num, r_all_M10, r_dict_M10, r_last_M10)

            x_pos = sample_nan_pos(x_num, r_dict_pos, r_dict_pos_withR) # 4) sample position

            attr = np.asarray([fill_panel(x_pos[i], x_shape[i], x_size[i], x_color[i]) for i in range(3)])
            attr_all.append(attr)

            del x_shape, x_size, x_color, x_pos, x_num 

    dur = time.time() - start_time
    
    attr_all = np.asarray(attr_all)
    a = np.copy(attr_all).reshape((-1, 3*9*3))
    b = np.unique(a, axis=0)
    ind = np.random.choice(np.arange(len(attr_all)), 12000,replace=False)
    np.save(data_dir+'aR/'+str(i_a)+'_'+str(i_R), attr_all[ind])
    
    print(str(i_R)+' -- '+str(dur)+' -- '+str(len(b) == len(a)))

0 -- 0.513812780380249 -- True
1 -- 0.8829183578491211 -- True
2 -- 0.591862678527832 -- True
3 -- 0.6046969890594482 -- True
4 -- 0.9286031723022461 -- True
5 -- 0.10526132583618164 -- True
6 -- 0.10176587104797363 -- True
